In [1]:
import pandas as pd
import glob
import csv
import pickle
import re
import tweepy
from time import strftime

In [5]:
file = glob.glob("U:/Twitter Research/Fake News Scraping/Fake News Tweet Sharing/User/*.data")

In [12]:
objects = []
user = []
j=0
for i in file:
    j=j+1
    with (open(i, "rb")) as openfile:
        user.append(i)
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

In [18]:
fullusers = objects

In [39]:
def list_to_dataframe(fullusers):
    tweet_id = [tweet.id_str for tweet in alltweets]
    tweet_created = [tweet.created_at for tweet in alltweets]
    tweet_text = [tweet.full_text for tweet in alltweets]

    tweet_hashtags = [tweet.entities['hashtags'] for tweet in alltweets]
    user_id = [tweet.user.id for tweet in alltweets]
    screen_name_df = [tweet.user.screen_name for tweet in alltweets]
    retweet_count =  [tweet.retweet_count for tweet in alltweets]
    favorite_count = [tweet.favorite_count for tweet in alltweets]
    lang = [tweet.lang for tweet in alltweets]
    source = [tweet.source for tweet in alltweets]


    retweet_text = []
    i=0
    for i in range(len(alltweets)):

        try:
            retweet_text.append(alltweets[i].retweeted_status.full_text) 
        except:
            retweet_text.append(None)
        i = i+1


    mod_tweet_hashtags = []
    total_hashtags = []

    for t in tweet_hashtags:
        tag = ""
        count = 0
        if (bool(t) == False):
            mod_tweet_hashtags.append('None')
            total_hashtags.append(count)
        else:
            for i in t:
                tag = tag + " " + i.get('text')
                count +=1
            mod_tweet_hashtags.append(tag)
            total_hashtags.append(count)

    mention = [tweet.entities['user_mentions'] for tweet in alltweets]
    mod_tweet_mention_sc = []
    mod_tweet_mention_id = []
    k = 0
    for t in mention:
        if (bool(t) == False):
            mod_tweet_mention_sc.append('None')
            mod_tweet_mention_id.append('None')
        else:
            mod_tweet_mention_sc.append(mention[k][0]['screen_name'])
            mod_tweet_mention_id.append(mention[k][0]['id_str'])
        k = k+1

    tweet_dict = {'Tweet_ID': tweet_id, 
                  'Tweet_Time': tweet_created, 
                  'Tweet_Text': tweet_text, 
                  'Retweet_Text': retweet_text,
                  'Tweet_HashTags': mod_tweet_hashtags,
                  'Total_Hashtags' : total_hashtags,
                  'Retweet_Count': retweet_count,
                  'Favorite_Count': favorite_count,
                  'Tweet_Source' : source,
                  'Retweet_Screen_Name':mod_tweet_mention_sc,
                  'Retweet_Account_ID':mod_tweet_mention_id,
                  'Tweet_Lang': lang,
                  'User_ID': user_id,
                  'User_ScreenName': screen_name_df
                  }  


    df = pd.DataFrame(tweet_dict) 
    df['Retweet_Source_ScreenName'] = 'None'
    df['Retweet'] = False



    for i in range(0, len(df)):
        if 'RT' == df['Tweet_Text'][i].split()[0]:
            df['Retweet_Source_ScreenName'][i] = df['Tweet_Text'][i].split()[1][1:-1]
            df['Retweet'][i] = True


    return user_info

In [40]:
datauser = list_to_dataframe(fullusers)

In [60]:
datauser['UserCreated'] =  pd.to_datetime(datauser['UserCreated'])

In [61]:
datauser['UserCreated'] = datauser['UserCreated'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [65]:
datauser = datauser.drop(columns = 'created', axis = 1)

In [67]:
datauser.to_csv('Fake_Accounts.csv')